# QRコード検出

Opencvのインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q opencv-python-headless

In [ ]:
%run setup_demo_env.py

In [ ]:
import cv2
import kachaka_api
import numpy as np
from IPython.display import Image, clear_output, display

In [ ]:
client = kachaka_api.aio.KachakaApiClient()

以下実行後、スマートフォンなどで https://pf-robotics.github.io/textcode/ にアクセスし、好きな英数字を入力して表示されたQRコードをカチャカの前カメラの前に近付けて下さい。

In [ ]:
qcd = cv2.QRCodeDetector()

async for image in client.front_camera_ros_compressed_image.stream():
    cv_image = cv2.imdecode(np.frombuffer(image.data, dtype=np.uint8), flags=1)
    decoded_info, corners, _ = qcd.detectAndDecode(cv_image)
    if corners is not None:
        cv_image = cv2.polylines(cv_image, corners.astype(int), True, (0, 0, 255), 2)
    if decoded_info != "":
        cv_image = cv2.putText(
            cv_image,
            decoded_info,
            corners[0][0].astype(int),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
            cv2.LINE_AA,
        )
    _, ret = cv2.imencode(
        ".jpg",
        cv2.resize(cv_image, (int(cv_image.shape[1] / 2), int(cv_image.shape[0] / 2))),
    )
    clear_output(wait=True)
    display(Image(data=ret, format="jpeg"))